# Object Detection with SSD
### Here we demostrate detection on example images using SSD with PyTorch

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
import torch.utils.data as data
import torchvision.transforms as transforms
from torch.utils.serialization import load_lua
import numpy as np
import cv2
if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

from ssd import build_ssd
# from models import build_ssd as build_ssd_v1 # uncomment for older pool6 model

## Build SSD300 in Test Phase
1. Build the architecture, specifyingsize of the input image (300),
    and number of object classes to score (21 for VOC dataset)
2. Next we load pretrained weights on the VOC0712 trainval dataset  

In [8]:
net = build_ssd('test', 300, 3)    # initialize SSD 21 classes 
# net.load_weights('weights/ssd300_mAP_77.43_v2.pth')
# net.load_weights('weights/ssd300_mAP_77.43_v2.pth')
weight_file = 'weights/ssd1166_bhjctrained_finalv2_20180314153338.pth'
net.load_weights(weight_file)

Loading weights into state dict...
Finished!


## Load Image 
### Here we just load a sample image from the VOC07 dataset 

In [9]:
# image = cv2.imread('./data/example.jpg', cv2.IMREAD_COLOR)  # uncomment if dataset not downloaded
%matplotlib inline
from matplotlib import pyplot as plt
from data import VOCDetection, VOCroot, AnnotationTransform
# here we specify year (07 or 12) and dataset ('test', 'val', 'train') 
# Change the default VOCroot that was imported to the VOCroot on my machine(the VOCdevkit directory)
# VOCroot = '/Users/keith.landry/code/ssd.pytorch/data/VOCdevkit/'
# testset = VOCDetection(VOCroot, [('2007', 'val')], None, AnnotationTransform())
# img_id = 60
# image = testset.pull_image(img_id)

path_to_bball_im = '/Users/keith.landry/data/internal-experiments/basketball/bhjc/20180123/images/left_cam/'
# bball_file = 'left_scene2_rot180_00896_crop3.png'
# bball_file = 'left_scene2_rot180_00792_crop2.png'
# bball_file = 'left_scene2_rot180_00792.png'
bball_file = 'left_scene2_rot180_00132.png' # 442, 531, 855

# path_to_bball_im = '/Users/keith.landry/Desktop/'
# bball_file = 'ghost.jpg'


# path_to_bball_im = '/Users/keith.landry/Documents/'
# bball_file = 'bballtest.jpg'

image = cv2.imread(path_to_bball_im + bball_file, cv2.IMREAD_COLOR)
print(image.shape)

rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# View the sampled input image before transform
plt.figure(figsize=(10, 10))
plt.imshow(rgb_image)
plt.show()


(1166, 1583, 3)


## Pre-process the input.  
#### Using the torchvision package, we can create a Compose of multiple built-in transorm ops to apply 
For SSD, at test time we use a custom BaseTransform callable to
resize our image to 300x300, subtract the dataset's mean rgb values, 
and swap the color channels for input to SSD300.

In [10]:
x = cv2.resize(image, (1166, 1166)).astype(np.float32)
# x = cv2.resize(image, (300, 300)).astype(np.float32)
# x -= (104, 117, 123)  #VOC means
# x -= (103, 100, 94)   #BHJC means

# plt.imshow(x.astype(int))
x = x[:, :, ::-1].copy()
x = x.astype(np.float32)

x = torch.from_numpy(x).permute(2, 0, 1)
x.shape

torch.Size([3, 1166, 1166])

## SSD Forward Pass
### Now just wrap the image in a Variable so it is recognized by PyTorch autograd

In [11]:
xx = Variable(x.unsqueeze(0))     # wrap tensor in Variable
if torch.cuda.is_available():
    xx = xx.cuda()
y = net(xx)  # passes image through pretrained network

before confidence layer: torch.Size([1, 512, 146, 146])


after confidence layer: torch.Size([1, 12, 146, 146])


before confidence layer: torch.Size([1, 1024, 73, 73])


after confidence layer: torch.Size([1, 18, 73, 73])
before confidence layer: torch.Size([1, 512, 37, 37])
after confidence layer: torch.Size([1, 18, 37, 37])
before confidence layer: torch.Size([1, 256, 19, 19])
after confidence layer: torch.Size([1, 18, 19, 19])
before confidence layer: torch.Size([1, 256, 17, 17])
after confidence layer: torch.Size([1, 12, 17, 17])
before confidence layer: torch.Size([1, 256, 15, 15])
after confidence layer: torch.Size([1, 12, 15, 15])
after softmax: Variable containing:
(  0   ,.,.) = 
  9.9982e-01  9.0816e-05  8.9569e-05
  9.9980e-01  1.2471e-04  7.6381e-05
  9.7422e-01  2.6184e-03  2.3158e-02
                 ⋮                  
  3.5105e-01  3.1585e-01  3.3310e-01
  3.5078e-01  3.2686e-01  3.2236e-01
  3.5624e-01  3.1945e-01  3.2431e-01
[torch.FloatTensor of size 1x129674x3]

priors: torch.Size([129674, 4])


## Parse the Detections and View Results
Filter outputs with confidence scores lower than a threshold 
Here we choose 60% 

In [17]:
# from data import VOC_CLASSES as labels
from data.bhjc20180123_bball.bhjc import CLASSES as labels
top_k = 10

plt.figure(figsize=(15, 15))
colors = plt.cm.hsv(np.linspace(0, 1, 21)).tolist()
plt.imshow(rgb_image)  # plot the image for matplotlib
currentAxis = plt.gca()

detections = y.data
# scale each detection back up to the image
scale = torch.Tensor(rgb_image.shape[1::-1]).repeat(2)
for i in range(detections.size(1)):
# for i in [2]:  # just label people
    j = 0
    while detections[0, i, j, 0] >= .5:
        score = detections[0, i, j, 0]
        label_name = labels[i-1]
        display_txt = '%s: %.2f'%(label_name, score)
        pt = (detections[0,i,j,1:]*scale).cpu().numpy()
        coords = (pt[0], pt[1]), pt[2]-pt[0]+1, pt[3]-pt[1]+1
        # print(label_name, score, coords)
        color = colors[i]
        currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor=color, linewidth=2))
        currentAxis.text(pt[0], pt[1], display_txt, bbox={'facecolor': color, 'alpha': 0.5})
        j += 1

In [55]:
coords

((621.33447, 798.11469), 6.69073486328125, 15.88861083984375)

In [7]:
net.priorbox.max_sizes

[30, 56, 81, 107, 132, 158]

In [16]:
detections.size(1)

3

In [8]:
net.priorbox.min_sizes

[1, 80, 80, 80, 80, 80]

In [23]:
detections


( 0 , 0 ,.,.) = 
   0   0   0   0   0
   0   0   0   0   0
   0   0   0   0   0
         ⋮          
   0   0   0   0   0
   0   0   0   0   0
   0   0   0   0   0

( 0 , 1 ,.,.) = 
   0   0   0   0   0
   0   0   0   0   0
   0   0   0   0   0
         ⋮          
   0   0   0   0   0
   0   0   0   0   0
   0   0   0   0   0

( 0 , 2 ,.,.) = 
   0   0   0   0   0
   0   0   0   0   0
   0   0   0   0   0
         ⋮          
   0   0   0   0   0
   0   0   0   0   0
   0   0   0   0   0
[torch.FloatTensor of size 1x3x200x5]

In [37]:
labels


('person', 'basketball')

In [24]:
any(detections[0, :, :, 1] != 0)

RuntimeError: bool value of non-empty torch.ByteTensor objects is ambiguous

In [73]:
net.priors

Variable containing:
 0.0034  0.0034  0.0257  0.0257
 0.0034  0.0034  0.0364  0.0364
 0.0034  0.0034  0.0364  0.0182
               ⋮                
 1.0000  1.0000  0.2473  0.2473
 1.0000  1.0000  0.3202  0.1601
 1.0000  1.0000  0.1601  0.3202
[torch.FloatTensor of size 129674x4]

In [79]:
a = np.array(net.priors[:, 3])